### Jumia Web Scraper

In [ ]:
import warnings
warnings.filterwarnings(action="ignore")

import re
import numpy as np
import pandas as pd

import chromedriver_binary
from bs4 import BeautifulSoup
from selenium import webdriver

In [ ]:
def main(keyword, pages_to_scrape, discount):
    # Startup the driver
    driver = webdriver.Chrome()

    # After observing the Jumia web page, the URL format was detected
    url = "https://www.jumia.com.ng/catalog/?q={}&price_discount={}-100&page={}#catalog-listing"

    # Generate a list of URLs based on the specified parameters
    url_list = [url.format(keyword, discount, i).replace(" ", "+") for i in range(1, pages_to_scrape + 1)]

    # Data dictionary to store scraped information
    data = {
        "Product_name": [],
        "Old_Price": [],
        "Current_Price": [],
        "Discount": [],
        "Rating": [],
        "No_of_Votes": [],
        "Image_link": []
    }

    # Iterate through each URL in the list
    for link in url_list:
        # Get the link from the driver
        driver.get(link)

        # Load BeautifulSoup
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Extract image links
        for image_link in soup.find_all(name="div", class_="img-c"):
            data["Image_link"].append(image_link.find("img").attrs["data-src"])

        # Extract the number of votes
        for votes in soup.find_all(name="div", class_="info"):
            text = votes.get_text()
            try:
                data["No_of_Votes"].append(re.search(r'\((\d+)\)', text).group(1))
            except:
                data["No_of_Votes"].append(np.nan)

        # Extract old prices
        for old_price in soup.find_all(name="div", class_="info"):
            data["Old_Price"].append(old_price.find(name="div", class_="old").text)

        # Extract current prices
        for current_price in soup.find_all(name="div", class_="info"):
            data["Current_Price"].append(current_price.find(name="div", class_="prc").text)

        # Extract discounts
        for discount in soup.find_all(name="div", class_="info"):
            data["Discount"].append(discount.find(name="div", class_="bdg _dsct _sm").text)

        # Extract product names
        for product_name in soup.find_all(name="div", class_="info"):
            data["Product_name"].append(product_name.find(name="h3", class_="name").text)

        # Extract product ratings
        for rating in soup.find_all(name="div", class_="info"):
            try:
                data["Rating"].append(rating.find(name="div", class_="rev").find(name="div", class_="stars _s").text)
            except:
                data["Rating"].append(np.nan)

    # Close the WebDriver
    driver.quit()

    # Convert the data dictionary to a pandas DataFrame and return it
    return pd.DataFrame(data)

In [ ]:
main(keyword="Iphone case", pages_to_scrape=5, discount=10)

,Product_name,Old_Price,Current_Price,Discount,Rating,No_of_Votes,Image_link
0,New IPhone 11 12 13 14 Pro Max 12 13 Mini Xs X...,"₦ 7,500","₦ 3,000",60%,4.4 out of 5,49,https://ng.jumia.is/unsafe/fit-in/300x300/filt...
1,New IPhone 11 12 13 14 Pro Max 12 13 Mini Xs X...,"₦ 7,500","₦ 3,000",60%,4 out of 5,15,https://ng.jumia.is/unsafe/fit-in/300x300/filt...
2,For IPhone X XS XSmax Seconds Change 11 PRO MA...,"₦ 3,500","₦ 2,000",43%,3.8 out of 5,8,https://ng.jumia.is/unsafe/fit-in/300x300/filt...
3,New IPhone 14 Pro Max Luxury Silicone Back Case,"₦ 8,000","₦ 3,000",63%,NaN,NaN,https://ng.jumia.is/unsafe/fit-in/300x300/filt...
4,New IPhone 11 12 13 14 Pro Max 12 13 Mini Xs X...,"₦ 7,500","₦ 3,000",60%,4 out of 5,4,https://ng.jumia.is/unsafe/fit-in/300x300/filt...
...,...,...,...,...,...,...,...
195,Phone Case for Apple iPhone 15 Pro Max Plating...,"₦ 5,235","₦ 2,751",47%,NaN,NaN,https://ng.jumia.is/unsafe/fit-in/300x300/filt...
196,"IPhone 15 Pro Max Case,Ultra Thin Electroplate...","₦ 15,918","₦ 7,959",50%,NaN,NaN,https://ng.jumia.is/unsafe/fit-in/300x300/filt...
197,For IPhone 6 Plus & 6s Plus 0.75mm Double-side...,"₦ 3,399","₦ 2,360",31%,4 out of 5,1,https://ng.jumia.is/unsafe/fit-in/300x300/filt...
198,IPhone 11 Pro Max Case Soft Hat Girl Phone Bac...,"₦ 3,500","₦ 2,200",37%,3 out of 5,2,https://ng.jumia.is/unsafe/fit-in/300x300/filt...


### Eddy